# Einfacher Spracherkenner

Idee: Untersuche die Verteilung von Buchstaben-Trigrammen.

Für jede zu erkennende Sprache wird ein Wikipediaartikel geladen. Die Buchstabenverteilung dort dient als Bezugsgröße.

Die Instanz `cv = CountVectorizer( analyzer='char', ngram_range=(3,3))` bereitet die Trigrammerstellung vor. Die Liste von Texten (Liste von Listen) in `doc` erzeugt mittels
`cv.fit_transform(doc)` die Verteilungsmatrix. Nach einer Transposition entspicht jede Spalte einer Sprache, jede Zeile einem Trigramm.

Der Test, in welcher Sprache ein Text verfasst ist, beruht auf einem Ähnlichkeitsvergleich zwischen Satz und Verteilungsmatrix. Dazu wird zunächst Text analog zur Verteilungsmatrix in einen Verteilungsvektor übeführt. Eine Normierung unterbleibt an dieser Stelle (siehe *Optimierungen*). Das Skalarprodukt aus jeweils einer Spalte der Verteilungsmatrix *dfi* und des Verteilungsvektors *q* ist ein Maß für die Übereinstimmung *cos(x)*:

cos(x) * |dfi| * |q| = dfi * q  <=>

cos(X) = dfi * q / (|dfi| * |q|)

$$
k(x, y) = \frac{x y^\top}{\|x\| \|y\|}
$$

Dafür gibt es auch eine Methode:

`from sklearn.metrics.pairwise import cosine_similarity`
`cosx = cosine_similarity([df.iloc[:, i].values], [q_vec]).item()`

## Optimierungen

Viele Optimierungsschitte fehlen:

* Aufbereiten der Texte, aus der die Verteilungsstatistik abgeleitet wird.
* Verkürzen der Verteilungsmatrix auf die Einträge, die ausreichend stark besetzt sind (7000 Trigramme sind unnötig)
* Rolle der Leerzeichen: helfen sie bei der Unterscheidung?

In [1]:
%reset -f
import re
import string
import requests
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import wikip
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
READDATA = False # load data from wikipedia otherwise local
fno = 'data/corpusD2s_deennlit.csv'
fnot = 'data/corpusD2st_deennlit.csv'

In [3]:
articles = ['https://de.wikipedia.org/wiki/Data_Science', 'https://en.wikipedia.org/wiki/Data_science', 
            'https://nl.wikipedia.org/wiki/Datawetenschap', 'https://it.wikipedia.org/wiki/Scienza_dei_dati']
articles_cc = ['DE', 'EN', 'NL', 'IT']

assert len(articles) == len(articles_cc), "misfit articles_cc and articles"

wiki_lst=[]
title=[]

if READDATA: 
    for article in articles:
       print("loading content: ",article)
       wiki_lst.append(wikip.get_wikipedia_text(article))
       title.append(article.split('/')[-1])
    corpusDs = pd.DataFrame({'article': wiki_lst})
    corpusDst = pd.DataFrame({'article': wiki_lst, 'title': title})
    corpusDs.to_csv(fno,  index = None)
    corpusDst.to_csv(fnot,  index = None)
else:
    corpusDs = pd.read_csv(fno)
    corpusDst = pd.read_csv(fnot)

Vorbereiten der Daten, um die Vergleichsstatistik für die Trigramme zu erzeugen.

In [4]:
# Prepare list of lists, each list one language sample.

docs = corpusDs['article'].tolist()

Erzeugen der normierten Verteilungsmatrix. Jede Spalte entspricht einer Sprache, jede Zeile einem Trigramm.

In [5]:
#cv = CountVectorizer( analyzer='char', ngram_range=(3,3))#'char_wb'
cv = TfidfVectorizer(use_idf=False , norm='l2', analyzer='char', ngram_range=(3,3))
X = cv.fit_transform(docs)
# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=cv.get_feature_names_out(), columns=articles_cc)
print(df.shape)
df.head()


(3811, 4)


,DE,EN,NL,IT
"""a",0.0,0.000000,0.0,0.004103
"""c",0.0,0.002988,0.0,0.000000
"""d",0.0,0.020914,0.0,0.000000
"""e",0.0,0.002988,0.0,0.000000
"""f",0.0,0.002988,0.0,0.000000


Nach Anpassung des Abfragesatzes `q_vec` wird verglichen:

Ähnlichkeit cos(x) =  dfi * q / (|dfi| * |q|)
    
`np.dot(df.iloc[:, i].values, q_vec) / np.linalg.norm(df.iloc[:, i]) * np.linalg.norm(q_vec)`  

In [21]:
def get_similar_articles(q, df, vec): 
    """
    calculate similariy between df (-Matrix) and q
    The query q may contain 
    
     input: query, TF-IDF-Matrix, TfidfVectorizer-instance
        convert inquiry to TF-IDF vector
        calculate  TF-IDF_article_i * transformed(q)
    output: dictionary{document_index: similarity}
    """
    no_articles = df.shape[1]
    print("query:", q)
    q = [q] #not normalized (otherwise the few values may be become too small)
    q_vec = vec.transform(q).toarray().reshape(df.shape[0],)
    sim = {}
    for i in range(4): #df.shape[1]):
        #sim[i] = np.dot(df.iloc[:, i].values, q_vec) / np.linalg.norm(df.iloc[:, i]) * np.linalg.norm(q_vec)
        sim[i] = cosine_similarity([df.iloc[:, i].values], [q_vec]).item()
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    return(sim_sorted)

def process_result(sim_sorted):
    """
    nice output of results
    input: dictionary{document_index: similarity}
        print(index, similarity, document title)
    """
    global corpusDst

    for k, v in sim_sorted:
        if v != 0.0:
            print(f'Similarity: {v:.4g} to {df.columns[k]}')

list_of_queries = ['aber ich kaufe lieber in Geschäften', 
                   'De in de index opgenomen aandelen vertegenwoordigen ongeveer',
                   'Inoltre, è stata introdotta una nuova condizione rigorosa']

for q in list_of_queries:
    process_result(get_similar_articles(q, df, cv ))
    print()



query: aber ich kaufe lieber in Geschäften
Similarity: 0.2133 to DE
Similarity: 0.115 to NL
Similarity: 0.0722 to EN
Similarity: 0.05532 to IT

query: De in de index opgenomen aandelen vertegenwoordigen ongeveer
Similarity: 0.396 to NL
Similarity: 0.2744 to DE
Similarity: 0.134 to EN
Similarity: 0.1174 to IT

query: Inoltre, è stata introdotta una nuova condizione rigorosa
Similarity: 0.226 to IT
Similarity: 0.1917 to EN
Similarity: 0.1135 to DE
Similarity: 0.11 to NL



## Vergleich der häufigsten Trigramme